# 프로젝트: 멋진 작사가 만들기


# 1. 필요한 라이브러리 불러오기

In [68]:
import os, re
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

# 2. 데이터 읽어오기


In [69]:
import glob

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

In [70]:
# 여러개의 txt 파일을 모두 읽어서 raw_corpus에 담기

for txt_file in txt_list:         
    with open(txt_file, "r") as f:       # 파일을 읽기모드로 열기.
        raw = f.read().splitlines()      # 라인 단위로 끊어서 list 형태로 읽어오기
        raw_corpus.extend(raw)           # raw_corpus에 넣기     

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])    # 3개의 라인 확인하기.

데이터 크기: 187088
Examples:
 ["Busted flat in Baton Rouge, waitin' for a train", "And I's feelin' near as faded as my jeans", 'Bobby thumbed a diesel down, just before it rained']


In [71]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue       # 길이가 0인 문장은 건너뛰기.
    
    if idx > 9: break                   # 일단 문장 10개만 확인하기.
    
    print(sentence)

Busted flat in Baton Rouge, waitin' for a train
And I's feelin' near as faded as my jeans
Bobby thumbed a diesel down, just before it rained
It rode us all the way to New Orleans I pulled my harpoon out of my dirty red bandanna
I was playin' soft while Bobby sang the blues, yeah
Windshield wipers slappin' time, I was holdin' Bobby's hand in mine
We sang every song that driver knew Freedom's just another word for nothin' left to lose
Nothin', don't mean nothin' hon' if it ain't free, no no
And, feelin' good was easy, Lord, when he sang the blues
You know, feelin' good was good enough for me


# 3. 데이터 정제

`preprocess_sentence()` 함수를 만들어서 데이터를 정제.
추가로 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거한다.
문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외.

In [72]:
# 토큰화(Tokenize)

# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지.


def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence


In [73]:
# 정제 데이터 구축

corpus = []

for sentence in raw_corpus:
    # 원하지 않는 문장을 건너뛰기.
    if len(sentence) == 0: continue    # 길이 0
    if len(sentence.split()) >= 12: continue # 15개 이하(start, end 포함)
    
    # 정제를 하고 담기.
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)

corpus[:10]  # 정제결과 확인

['<start> busted flat in baton rouge , waitin for a train <end>',
 '<start> and i s feelin near as faded as my jeans <end>',
 '<start> bobby thumbed a diesel down , just before it rained <end>',
 '<start> i was playin soft while bobby sang the blues , yeah <end>',
 '<start> windshield wipers slappin time , i was holdin bobby s hand in mine <end>',
 '<start> nothin , don t mean nothin hon if it ain t free , no no <end>',
 '<start> and , feelin good was easy , lord , when he sang the blues <end>',
 '<start> you know , feelin good was good enough for me <end>',
 '<start> there bobby shared the secrets of my soul <end>',
 '<start> through all kinds of weather , through everything we done <end>']

# 4. 평가 데이터셋 분리
***
**훈련데이터와 평가 데이터를 분리하기**  

tokenize() 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 하겠습니다. 단어장의 크기는 12,000 이상 으로 설정하세요! 총 데이터의 20% 를 평가 데이터셋으로 사용해 주세요!

In [74]:
# tokenize() 함수로 데이터를 Tensor로 변환

def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
            num_words= 12000, #단어장의 크기:12000 이상
            filters=' ',
            oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)
    

[[   2 3918 1853 ...    0    0    0]
 [   2    8    5 ...    0    0    0]
 [   2  849 6699 ...    0    0    0]
 ...
 [   2    5   23 ...    0    0    0]
 [   2    5   23 ...    0    0    0]
 [   2    5   23 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f28cef4caf0>


In [75]:
# 단어사전 구축 인덱스 확인
for idx in tokenizer.index_word:
    print(idx,":", tokenizer.index_word[idx])
    
    if idx >= 10: break
        
#생성된 텐서를 소스와 타겟으로 분리해 모델 학습

src_input = tensor[:, :-1]       #소스 문장을 생성
tgt_input = tensor[:, 1:]        #타겟 문장을 생성

print(src_input[0])
print(tgt_input[0])

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to
[   2 3918 1853   14    1 4369    4 1322   28    9  656    3    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0]
[3918 1853   14    1 4369    4 1322   28    9  656    3    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0]


In [76]:
#데이터셋 객체 생성
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input)  // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words +1

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset


<BatchDataset shapes: ((256, 29), (256, 29)), types: (tf.int32, tf.int32)>

In [77]:
# 총 데이터의 20%를 평가 데이터셋으로 만들기

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input,
                                                          test_size=0.2,
                                                          shuffle=True,
                                                          random_state=12
                                                         )

# 5. 인공지능 만들기
***

모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 `val_loss` 값을 2.2 수준으로 줄일 수 있는 모델을 설계하세요! (Loss는 아래 제시된 Loss 함수를 그대로 사용!)

그리고 멋진 모델이 생성한 가사 한 줄을 제출하시길 바랍니다!

In [78]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256                 # 워드 벡터의 차원수, 즉 단어가 추상적으로 표현되는 크기
hidden_size = 1024                   # 모델에 얼마나 많은 일꾼을 둘 것인지.
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [79]:
# 데이터셋에서 데이터 한 배치만 불러오기
for src_sample, tgt_sample in dataset.take(1): break

model(src_sample)


<tf.Tensor: shape=(256, 29, 12001), dtype=float32, numpy=
array([[[ 1.98183276e-04,  1.10739304e-04,  1.50641514e-04, ...,
         -7.86882374e-05, -3.52473508e-05,  9.43987179e-05],
        [ 3.91557514e-05,  1.60865038e-05,  9.94312795e-05, ...,
         -1.77722177e-04, -2.37491229e-04,  1.11649570e-04],
        [-1.47601910e-04, -2.40673253e-04,  3.54548902e-05, ...,
         -3.41957522e-04, -4.39740514e-04, -8.09758640e-05],
        ...,
        [ 1.48985570e-03, -1.35596230e-04, -8.59143693e-05, ...,
         -1.89919746e-03,  2.09501100e-04,  6.66902051e-05],
        [ 1.49437541e-03, -7.36683651e-05, -1.48865787e-04, ...,
         -1.93941174e-03,  2.51097284e-04,  4.80871931e-05],
        [ 1.49734819e-03, -1.77659876e-05, -2.05424600e-04, ...,
         -1.97252841e-03,  2.89809686e-04,  3.09579118e-05]],

       [[ 1.98183276e-04,  1.10739304e-04,  1.50641514e-04, ...,
         -7.86882374e-05, -3.52473508e-05,  9.43987179e-05],
        [ 4.02361096e-04,  2.05652934e-04,  1

In [80]:
model.summary()

Model: "text_generator_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  3072256   
_________________________________________________________________
lstm_2 (LSTM)                multiple                  5246976   
_________________________________________________________________
lstm_3 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense_1 (Dense)              multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [81]:
# 모델 학습하기
history= []
epochs = 10

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
history = model.fit(enc_train,
                    dec_train,
                    epochs=epochs,
                    batch_size=256,
                    validation_data=(enc_val,dec_val)
                    ,verbose=1)


Epoch 1/10
476/476 [==============================] - 188s 390ms/step - loss: 1.7567 - val_loss: 1.5098
Epoch 2/10
476/476 [==============================] - 198s 416ms/step - loss: 1.4518 - val_loss: 1.4186
Epoch 3/10
476/476 [==============================] - 199s 417ms/step - loss: 1.4031 - val_loss: 1.3811
Epoch 4/10
476/476 [==============================] - 199s 419ms/step - loss: 1.3281 - val_loss: 1.3381
Epoch 5/10
476/476 [==============================] - 200s 419ms/step - loss: 1.2779 - val_loss: 1.3077
Epoch 6/10
476/476 [==============================] - 200s 420ms/step - loss: 1.2339 - val_loss: 1.2823
Epoch 7/10
476/476 [==============================] - 200s 420ms/step - loss: 1.1929 - val_loss: 1.2617
Epoch 8/10
476/476 [==============================] - 200s 419ms/step - loss: 1.1548 - val_loss: 1.2433
Epoch 9/10
476/476 [==============================] - 199s 419ms/step - loss: 1.1191 - val_loss: 1.2279
Epoch 10/10
476/476 [==============================] - 200s 420m

In [82]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    while True:
        # 입력받은 문장의 텐서를 입력
        predict = model(test_tensor) 
        # 예측된 값 중 가장 높은 확률인 word index를 뽑기
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 예측된 word index를 문장 뒤에 붙임
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마침
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [84]:
#문장 생성 함수 실행
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , liberian girl <end> '

# 회고
***
이번 EXPLORATION 6번을 하면서 내용이 이해가 안되는 것이 많았다.  
시퀀스 데이터부터 데이터를 다듬고 학습시키고 실제로 문장을 생성하는 것 까지 생소했던 것 같다.  
전처리하는 과정에서 토큰화, 텐서를 하는 과정이 중요하고 시간도 걸리는 것 같다.  
데이터 정제하는 과정에 따라 모델이 학습하고 성능이 달라지는 것을 확인하였다.  
토큰의 개수를 정하는 이유와 tokenize 함수에 대한 부분도 좀 더 이해가 필요한 것 같다.  
실제로 학습시키고 val_loss 값이 1.2152로 비교적 잘 학습이 된 것같다.  
문장을 생성하는 함수를 실행했을때  
'liberian girl'이란 단어가 나왔는데 마이클 잭슨의 노래에 있는 가사라는 것을 확인하였다.  
모델이 문장 생성을 잘 했다고 생각하지만, 쉽지 않은 실습이였던 것 같다.  